# Copyright

Jelen iPython notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott "Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült, a https://github.com/bradleypallen/keras-movielens-cf oldalon található példák és minták alapján (MIT licensz).

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot GPLv3 licensz védi. Újrafelhasználás esetén lehetőség szerint kérjük az alábbi szerzőt értesíteni.

2016 (c) Szaszák György (szaszak kukac tmit pont bme pont hu)

In [1]:
import pandas as pd
import numpy as np
from CFModel import CFModel

Using Theano backend.


## Konstansok (előre elkészített adatállományok és a beágyazás dimenziószáma), valamint egy előtanított modell


In [2]:
RATINGS_CSV_FILE = 'ml1m_ratings.csv'
USERS_CSV_FILE = 'ml1m_users.csv'
MOVIES_CSV_FILE = 'ml1m_movies.csv'
MODEL_WEIGHTS_FILE = 'ml1m_weights.h5'
EMBEDDING_DIM = 120

## MovieLens 1M betöltése

In [3]:
ratings = pd.read_csv(RATINGS_CSV_FILE, sep='\t', encoding='latin-1', usecols=['userid', 'movieid', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()
print (len(ratings), 'értékelés betöltve.')

1000209 értékelés betöltve.


In [4]:
users = pd.read_csv(USERS_CSV_FILE, sep='\t', encoding='latin-1', usecols=['userid', 'gender', 'zipcode', 'age_desc', 'occ_desc'])
print (len(users), 'felhasználó a', max_userid, 'felhasználó közül betöltve.')

6040 felhasználó a 6040 felhasználó közül betöltve.


In [5]:
movies = pd.read_csv(MOVIES_CSV_FILE, sep='\t', encoding='latin-1', usecols=['movieid', 'title', 'genre'])
print (len(movies), 'film a', max_movieid, 'film közül betöltve.')

3883 film a 3952 film közül betöltve.


## Filmajánlás egy véletlenszerűen választott felhasználónak
A felhasználói értékelést jelezzük előre a filmekre.

In [6]:
# keras.layers.embeddings.Embedding(input_dim, output_dim, init='uniform', input_length=None, W_regularizer=None, activity_regularizer=None, W_constraint=None, mask_zero=False, weights=None, dropout=0.0)
# 2 ágon torténik közös embedding: userekre és itemekre, K_FACTORS dimenzióra
trained_model = CFModel(max_userid, max_movieid, EMBEDDING_DIM)

In [7]:
# 2 súlymátrixra van szükségünk, egy user->embeddingre és egy item->embeddingre
trained_model.load_weights(MODEL_WEIGHTS_FILE)

In [8]:
# Egy véletlen felhasználó és adatai
test_user=np.random.randint(0,max_userid)
users[users['userid'] == test_user]

,userid,gender,zipcode,age_desc,occ_desc
5679,5680,M,94117,50-55,writer


In [10]:
# Egy véletlen film és adatai
test_movie=np.random.randint(0,max_movieid)
movies[movies['movieid'] == test_movie]

,movieid,title,genre
2366,2435,Hurlyburly (1998),Drama


In [9]:
# Ez a metódus model.predict-et futtat, indexelést átképezzük (adatbázisban 1-től indul)
def predict_rating(userid, movieid):
    return trained_model.rate(userid - 1, movieid - 1)

In [11]:
# Betöltjük a felhasználói értékeléseket
user_ratings = ratings[ratings['userid'] == test_user][['userid', 'movieid', 'rating']]
# Hozzáadunk egy ajánló oszlopot és minden filmre generáljuk, hogy mennyire passzol a felhasználó ízléséhez
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(test_user, x['movieid']), axis=1)
# Kiiratáshoz a predikciós és filmes táblákat egyesítjük a filmazonosító (ez közös) mentén
# Első 10-et írjuk ki
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(movies, 
                                                on='movieid', 
                                                how='inner', 
                                                suffixes=['_u', '_m']).head(10)

,userid,movieid,rating,prediction,title,genre
0,5680,1408,5,3.986941,"Last of the Mohicans, The (1992)",Action|Romance|War
1,5680,1284,5,4.547963,"Big Sleep, The (1946)",Film-Noir|Mystery
2,5680,1264,5,4.593738,Diva (1981),Action|Drama|Mystery|Romance|Thriller
3,5680,3176,5,4.477126,"Talented Mr. Ripley, The (1999)",Drama|Mystery|Thriller
4,5680,1267,5,4.743114,"Manchurian Candidate, The (1962)",Film-Noir|Thriller
5,5680,3134,5,5.227365,"Grand Illusion (Grande illusion, La) (1937)",Drama|War
6,5680,2186,5,4.789973,Strangers on a Train (1951),Film-Noir|Thriller
7,5680,750,5,4.875195,Dr. Strangelove or: How I Learned to Stop Worr...,Sci-Fi|War
8,5680,1289,5,4.525403,Koyaanisqatsi (1983),Documentary|War
9,5680,1262,5,4.528728,"Great Escape, The (1963)",Adventure|War


In [12]:
# Szűrés azokra a filmekre, amelyeket a felhasználónk még nem látott
recommendations = ratings[ratings['movieid'].isin(user_ratings['movieid']) == False][['movieid']].drop_duplicates()
# Ajánlás a még nem látott filmekre
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(test_user, x['movieid']), axis=1)
# Kiírjuk az első 10-et
recommendations.sort_values(by='prediction',
                          ascending=False).merge(movies,
                                                 on='movieid',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(10)

,movieid,prediction,title,genre
0,2351,5.208909,Nights of Cabiria (Le Notti di Cabiria) (1957),Drama
1,669,5.177761,Aparajito (1956),Drama
2,1423,5.150979,Hearts and Minds (1996),Drama
3,668,5.130191,Pather Panchali (1955),Drama
4,3092,5.117554,Chushingura (1962),Drama
5,3415,5.074968,"Mirror, The (Zerkalo) (1975)",Drama
6,3338,5.067842,For All Mankind (1989),Documentary
7,2905,5.056692,Sanjuro (1962),Action|Adventure
8,3030,5.036850,Yojimbo (1961),Comedy|Drama|Western
9,3634,5.036220,Seven Days in May (1964),Thriller
